In [1]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15965, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 15965 (delta 83), reused 108 (delta 70), pack-reused 15831
Receiving objects: 100% (15965/15965), 14.60 MiB | 6.91 MiB/s, done.
Resolving deltas: 100% (10956/10956), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.5/593.5 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
/content/yolov5


In [2]:
%cd /content
!curl -L "https://public.roboflow.com/ds/8vL0EultzZ?key=Fnp15aVt1A" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Streaming output truncated to the last 5000 lines.
 extracting: export/labels/1478900109306048491_jpg.rf.7f8266ddc28a351f0dcd20604c268259.txt  
 extracting: export/labels/1478900109306048491_jpg.rf.y4x0eVX2sO7ecR0TUaVr.txt  
 extracting: export/labels/1478900109878689818_jpg.rf.VDdJVseJIvOB61vpFirp.txt  
 extracting: export/labels/1478900109878689818_jpg.rf.bcebbb0d35bd86bda671f535b3f64031.txt  
 extracting: export/labels/1478900110449179205_jpg.rf.996b317597119cdcc563966e7c02b624.txt  
 extracting: export/labels/1478900110449179205_jpg.rf.BeHDkSoCfqcbL7l2e86o.txt  
 extracting: export/labels/1478900111020889294_jpg.rf.dac0676c053b218573c57367055fe8b2.txt  
 extracting: export/labels/1478900111020889294_jpg.rf.uUaiVa1Rn8K5eJTu2pvC.txt  
 extracting: export/labels/1478900111592085926_jpg.rf.58ddab1db87104355311ee1696d35449.txt  
 extracting: export/labels/1478900111592085926_jpg.rf.nA12WQhyH7O7E7ggQ4nn.txt  
 extracting: export/labels/1478900112162597702_jpg.rf.0fcefae4646f6879fce730488

In [3]:
!mkdir train/
!mkdir train/images
!mkdir train/labels
!mkdir test/
!mkdir test/images
!mkdir test/labels
!mkdir valid/
!mkdir valid/images
!mkdir valid/labels

In [11]:
#copying the images from export to test, train and valid
#We are basically dividing the data into 20(test) 80(train) percent
import os
import shutil

allImages = []
for filename in os.listdir("export/images/"):
  if filename.endswith(".jpg"):
    allImages.append(filename)
trainImages = []
valImages = []
trainLabels = []
valLabels = []
testImages = []
testLabels = []
print(len(allImages))
testCount = int(0.2*len(allImages)/10)
trainCount = len(allImages)/10-testCount-1
valCount = int(0.2*trainCount)
trainCount = trainCount - valCount-1
print(trainCount,valCount,testCount)

#Creating a Dataset
j=0
k=0
for i in range(0,int(len(allImages)/10)):
  if i<trainCount:
    trainImages.append(allImages[i])
  elif i>=trainCount and i<(valCount + trainCount):
    valImages.append(allImages[i])
    #j+=1
  else:
    testImages.append(allImages[i])
    #k+=1
print(len(trainImages),len(valImages),len(testImages))

#Moving training images
source = "export/images/"
destination = "train/images/"
for fileT in trainImages:
  shutil.move(source+fileT, destination+fileT)

#Moving Validate images
source = "export/images/"
destination = "valid/images/"
for fileT in valImages:
  shutil.move(source+fileT, destination+fileT)

#Moving Testing images
source = "export/images/"
destination = "test/images/"
for fileT in testImages:
  shutil.move(source+fileT, destination+fileT)

#Moving Training labels
source = "export/labels/"
destination = "train/labels/"
for fileT in trainImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT, destination+fileT)

#Moving Validate Labels
source = "export/labels/"
destination = "valid/labels/"
for fileT in valImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT, destination+fileT)

#Moving Testing Labels
source = "export/labels/"
destination = "test/labels/"
for fileT in testImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT, destination+fileT)

14350
917.0 229 287
917 229 289


In [12]:
#Extracting information from the roboflow file
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']

In [13]:
#define number of classes based on data.yaml
import yaml
with open("data.yaml",'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

In [14]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]

In [15]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line ,'w') as f:
    f.write(cell.format(**globals()))

In [16]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

#parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33 #model depth multiple
width_multiple: 0.50 #layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [17]:
# train yolov5s on custom data for 50 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 20 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_

In [21]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7273488 parameters, 0 gradients
image 1/3428 /content/test/images/1478019954186238236_jpg.rf.c963c5ba70a979d12d52f9ca22bbe8d1.jpg: 288x416 3 cars, 44.1ms
image 2/3428 /content/test/images/1478019954685370994_jpg.rf.b2725ec9e703c9644495d02cbd07b86d.jpg: 288x416 3 cars, 7.7ms
image 3/3428 /content/test/images/1478019957180061202_jpg.rf.S2R8kzN3TeZTrk9rkjrm.jpg: 288x416 3 cars

In [24]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/inference/output/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

In [23]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --source ../video3.mp4 --conf 0.4

detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../video3.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-175-g5f11555 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7273488 parameters, 0 gradients
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 261, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 256, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5